# ReLU vs SELU networks on the SVHN data set

Adapted from SNNs bioinf-jku [SNNs bioinf-jku](https://github.com/bioinf-jku/SNNs)

## Fetch Dataset

In [ ]:
import os
import pickle
import sys
import tarfile
import zipfile

import matplotlib.pyplot as plt
import scipy.io

from urllib.request import urlretrieve

import time

import numpy as np

url = 'http://ufldl.stanford.edu/housenumbers/'
last_percent_reported = None

def get_data_set(name="train"):
    x = None
    y = None
    
    if name is "train":
        train_matfile = maybe_download('./data_set/train_32x32.mat')
        train_data = scipy.io.loadmat('./data_set/train_32x32.mat', variable_names='X').get('X')
        train_labels = scipy.io.loadmat('./data_set/train_32x32.mat', variable_names='y').get('y')
        
        x = train_data
        y = train_labels
        y[y == 10] = 0      
        
        y = dense_to_one_hot(y)
        
        x_mean_train = x.mean()
        x_sdev_train = x.std()
        x_var_train = x.var()
        
        x = (x - x_mean_train) / x_sdev_train
        x = x.reshape(32*32*3, 73257)
        x = np.transpose(x)
        
    if name is "test":
        test_matfile = maybe_download('./data_set/test_32x32.mat')
        test_data = scipy.io.loadmat('./data_set/test_32x32.mat', variable_names='X').get('X')
        test_labels = scipy.io.loadmat('./data_set/test_32x32.mat', variable_names='y').get('y')
        
        xt = test_data
        yt = test_labels
        yt[yt == 10] = 0
        
        y = dense_to_one_hot(yt)
        
        x_mean_test = xt.mean()
        x_sdev_test = xt.std()
        x_var_test = xt.var()
        
        x = (xt - x_mean_test) / x_sdev_test
        x = x.reshape(32*32*3, 26032)
        x = np.transpose(x)

    return x, y

def dense_to_one_hot(labels_dense, num_classes=10):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1

    return labels_one_hot

def _print_download_progress(count, block_size, total_size):
    pct_complete = float(count * block_size) / total_size
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)
    sys.stdout.write(msg)
    sys.stdout.flush()

def maybe_download(filename, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename, reporthook=_print_download_progress)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  return filename


## Scaled ELU

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops


def selu(x, name="selu"):
    """ When using SELUs you have to keep the following in mind:
    # (1) scale inputs to zero mean and unit variance
    # (2) use SELUs
    # (3) initialize weights with stddev sqrt(1/n)
    # (4) use SELU dropout
    """
    with ops.name_scope(name) as scope:
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
        return scale * tf.where(x >= 0.0, x, alpha * tf.nn.elu(x))

## Some helpers to build the network

In [ ]:
from math import sqrt
import numpy as np
import tensorflow as tf


def _variable_with_weight_decay(name, shape, activation, stddev, wd=None):    
    # Determine number of input features from shape
    f_in = np.prod(shape[:-1]) if len(shape) == 4 else shape[0]
    # Determine number of ouput features from shape
    f_out = shape[-1]
    
    # Calculate sdev for initialization according to activation function
    if activation == selu:
        sdev = sqrt(1 / f_in)
#         sdev = sqrt(2 / f_in)# He Initialization
#         sdev = sqrt(2 / (f_in+f_out)) #Xavier Initialization
    elif activation == tf.nn.relu:
#         sdev = sqrt(1 / f_in)
        sdev = sqrt(2 / f_in)# He Initialization
#         sdev = sqrt(2 / (f_in+f_out)) #Xavier Initialization
    elif activation == tf.nn.elu:
        sdev = sqrt(1.5505188080679277 / f_in)
    else:
        sdev = stddev
    
    var = tf.get_variable(name=name, shape=shape,
                          initializer=tf.truncated_normal_initializer(stddev=sdev, dtype=tf.float32))
    
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

In [ ]:
import tensorflow as tf


def conv2d(scope_name, input, activation, ksize, f_in, f_out, bias_init=0.0, stddev=5e-2):
    with tf.variable_scope(scope_name) as scope:
        kernel = _variable_with_weight_decay('weights', shape=[ksize, ksize, f_in, f_out], activation=activation,
                                             stddev=stddev)
        conv = tf.nn.conv2d(input, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.get_variable('biases', [f_out], initializer=tf.constant_initializer(bias_init), dtype=tf.float32)
        pre_activation = tf.nn.bias_add(conv, biases)
        return activation(pre_activation, name=scope.name)


def fc(scope_name, input, activation, n_in, n_out, stddev=0.04, bias_init=0.0, weight_decay=None):
    with tf.variable_scope(scope_name) as scope:
        weights = _variable_with_weight_decay('weights', shape=[n_in, n_out], activation=activation, stddev=stddev,
                                              wd=weight_decay)
        biases = tf.get_variable(name='biases', shape=[n_out], initializer=tf.constant_initializer(bias_init),
                                 dtype=tf.float32)
        return activation(tf.matmul(input, weights) + biases, name=scope.name)

## Build the model with a specified activation function

In [ ]:
def model(activation):
    _IMAGE_SIZE = 32
    _IMAGE_CHANNELS = 3
    _NUM_CLASSES = 10
    _RESHAPE_SIZE = 4 * 4 * 128
    
    # set activation function
    act = selu if activation == "selu" else tf.nn.elu if activation == "elu" else tf.nn.relu
    
    with tf.variable_scope(activation):
        # input
        with tf.name_scope('data'):
            x = tf.placeholder(tf.float32, shape=[None, _IMAGE_SIZE * _IMAGE_SIZE * _IMAGE_CHANNELS], name='Input')
            y = tf.placeholder(tf.float32, shape=[None, _NUM_CLASSES], name='Output')
            x_image = tf.reshape(x, [-1, _IMAGE_SIZE, _IMAGE_SIZE, _IMAGE_CHANNELS], name='images')
        
        # Conv 1
        conv1 = conv2d("conv1", input=x_image, activation=act, ksize=5, f_in=3, f_out=64)
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
        
        # Conv 2
        conv2 = conv2d("conv2", input=pool1, activation=act, ksize=5, f_in=64, f_out=64, bias_init=0.1)
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        
        # Conv 3-5
        conv3 = conv2d("conv3", input=pool2, activation=act, ksize=3, f_in=64, f_out=128)
        conv4 = conv2d("conv4", input=conv3, activation=act, ksize=3, f_in=128, f_out=128)
        conv5 = conv2d("conv5", input=conv4, activation=act, ksize=3, f_in=128, f_out=128)
        
        # Pool
        pool3 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool3')
        
        # Reshape
        reshape = tf.reshape(pool3, [-1, _RESHAPE_SIZE])
        dim = reshape.get_shape()[1].value
        
        # Fully Connected L2 Penalty
        fc1 = fc('fully_connected1', input=reshape, activation=act, n_in=dim, n_out=384, stddev=0.04, bias_init=0.1,
                 weight_decay=0.004)
        fc2 = fc('fully_connected2', input=fc1, activation=act, n_in=384, n_out=192, stddev=0.04, bias_init=0.1,
                 weight_decay=0.004)

        # Fully Connected Dropout
        #   fc1 = fc('fully_connected1', input=reshape, activation=act, n_in=dim, n_out=384, stddev=0.04, bias_init=0.1)
        
        #   keep_prob = tf.placeholder(tf.float32)
        #   fc1_drop = tf.nn.dropout(fc1, keep_prob)
    
        #   fc2 = fc('fully_connected2', input=fc1_drop, activation=act, n_in=384, n_out=192, stddev=0.04, bias_init=0.1)
        #   fc2_drop = tf.nn.dropout(fc2, keep_prob)
        
        # Softmax
        with tf.variable_scope('output') as scope:
            #     Uncomment when using L2
            weights = _variable_with_weight_decay('weights', [192, _NUM_CLASSES], stddev=1 / 192.0,
                                                  activation=activation,
                                                  wd=0.0)
            #   Uncomment when using dropout
            #   weights = _variable_with_weight_decay('weights', [192, _NUM_CLASSES], stddev=1 / 192.0,
            #                                        activation=activation)
            biases = tf.get_variable(name='biases', shape=[_NUM_CLASSES], initializer=tf.constant_initializer(0.0),
                                     dtype=tf.float32)
            softmax_linear = tf.add(tf.matmul(fc2, weights), biases, name=scope.name)
            
            # output
            y_pred_cls = tf.argmax(softmax_linear, dimension=1)
        
        # Define Loss and Optimizer
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=softmax_linear, labels=y))
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)
        # optimizer = tf.train.MomentumOptimizer(learning_rate=1e-4, momentum=0.9).minimize(loss)
        
        correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, dimension=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        # tf.summary.scalar("Accuracy/train", accuracy)

    #     Uncomment when using L2
    return {"x": x, "y": y, "output": y_pred_cls, "loss": loss, "accuracy": accuracy, "optimizer": optimizer, 
            "name": activation}

    #     Uncomment when using dropout
#     return {"x": x, "y": y, "output": y_pred_cls, "loss": loss, "accuracy": accuracy, "optimizer": optimizer, 
#             "name": activation, "keep_prob": keep_prob}


## Evaluate on Test Set

In [ ]:
def predict_test(test_x, test_y, models):
    """
        Make prediction for all images in test_x
    """
    i = 0
    predicted_class = {"selu": np.zeros(shape=len(test_x), dtype=np.int), 
                       #"elu": np.zeros(shape=len(test_x), dtype=np.int), 
                       "relu":np.zeros(shape=len(test_x), dtype=np.int)}
    while i < len(test_x):
        j = min(i + _BATCH_SIZE, len(test_x))
        batch_xs = test_x[i:j, :]
        batch_ys = test_y[i:j, :]
        for name, model in models.items():
            predicted_class[name][i:j] = sess.run(model["output"], feed_dict={
#                 model['x']: batch_xs, model['y']: batch_ys, model['keep_prob']: 1.0})
                model['x']: batch_xs, model['y']: batch_ys})
        i = j
    
    accuracy = {"selu": 0, "relu": 0}
    for name, model in models.items():
        correct = (np.argmax(test_y, axis=1) == predicted_class[name])        
        accuracy[name] = correct.mean() * 100        
    
    print("Accuracy on Test-Set (SELU/RELU): {0:.2f}% | {1:.2f}% |".format(
        accuracy["selu"], accuracy["relu"]))
    
    return accuracy


## Plotting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def plot_metric(title, ylabel, metric):
    # Training Accuracy
    plt.figure()    
    plt.title(title, size="xx-large")
    plt.ylabel(ylabel, size="x-large")    
    plt.tick_params(axis="x", bottom="off", labelbottom="off")
    
    # select manually for consistent colors
    plt.plot(metric["selu"], label="SELU", linewidth=2)
    #plt.plot(metric["elu"], label="ELU", linewidth=2)
    plt.plot(metric["relu"], label="RELU", linewidth=2)
        
    plt.legend()
    plt.show()

def plot(train_loss, train_accuracy, test_accuracy):    
    # Training Loss
    plot_metric("Training Loss", "Loss", train_loss)
    
    # Training Accuracy
    plot_metric("Training Accuracy", "Accuracy", train_accuracy)
    
    # Test Accuracy
    plot_metric("Test Accuracy", "Accuracy", test_accuracy)

## Training

In [ ]:
def train(session, num_epoch, train_x, train_y, test_x, test_y, models):
    """
        Train CNN
    """    
    train_loss = {"selu": [], "relu": []}
    train_accuracy = {"selu": [], "relu": []}    
    test_accuracy = {"selu": [], "relu": []}
    
    # start training
    print(time.localtime())
    for epoch in range(num_epoch):
        total_batch = int(train_x.shape[0]/_BATCH_SIZE)
        print("Epoch: ", epoch) 
        # Loop over all batches
        for i in range(total_batch):
            randidx = np.random.randint(len(train_x), size=_BATCH_SIZE)
            batch_xs = train_x[randidx]
            batch_ys = train_y[randidx]

            optimizers = []
            feed_dict = {}
            for name, model in models.items():
                optimizers.append(model["optimizer"])
#                 feed_dict.update({model["x"]: batch_xs, model["y"]: batch_ys, model["keep_prob"]: 0.5})
                feed_dict.update({model["x"]: batch_xs, model["y"]: batch_ys})

            # train
            session.run( optimizers, feed_dict=feed_dict)

            # print training loss
            if (i % 10 == 0) :
                l_selu, l_relu, acc_selu, acc_relu = session.run(
                    [models['selu']['loss'],  models['relu']['loss'], 
                     models['selu']['accuracy'], models['relu']['accuracy']],
                    feed_dict=feed_dict)

                msg = "Minibatch: {0:>6}, " \
                      "accuracy (SELU/RELU): {1:>6.1%} | {2:>6.1%} |, " \
                      "loss (SELU/RELU): {3:.2f} | {4:.2f} |"
                print(msg.format(i, acc_selu, acc_relu, l_selu, l_relu))            

                # collect metrics for plots                            
                train_loss["selu"].append(l_selu)
#                 train_loss["elu"].append(l_elu)
                train_loss["relu"].append(l_relu)
                train_accuracy["selu"].append(acc_selu)
#                 train_accuracy["elu"].append(acc_elu)
                train_accuracy["relu"].append(acc_relu)

        acc = predict_test(test_x, test_y, models)                
        test_accuracy["selu"].append(acc["selu"])
#         test_accuracy["elu"].append(acc["elu"])
        test_accuracy["relu"].append(acc["relu"])
        saver.save(session, save_path=_SAVE_PATH + "/checkpoint", global_step=epoch)
        print("Saved checkpoint.")
        
    print(time.localtime())
    return train_loss, train_accuracy, test_accuracy

## Parameters

In [ ]:
_IMG_SIZE = 32
_NUM_CHANNELS = 3
# _BATCH_SIZE = 128
_BATCH_SIZE = 500
_CLASS_SIZE = 10
# _ITERATION = 10000
_EPOCH = 50
_SAVE_PATH = "./checkpoints/svhn-he"

In [ ]:
import os

# Set GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

if not os.path.exists(_SAVE_PATH):
    os.makedirs(_SAVE_PATH)

## Create Models

In [ ]:
import tensorflow as tf

# Build Graph
relu = model("relu")
selu = model("selu")
#elu = model("elu")

## Run Training

In [ ]:
import tensorflow as tf

# Some Tensorflow configuration
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

# Initialize Dataset
train_x, train_y = get_data_set("train")
test_x, test_y = get_data_set("test")

saver = tf.train.Saver()
with tf.Session(config=tf_config) as sess:
    try:
        print("Trying to restore last checkpoint ...")
        last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
        saver.restore(sess, save_path=last_chk_path)
        print("Restored checkpoint from:", last_chk_path)
    except:
        print("Failed to restore checkpoint. Initializing variables instead.")
        sess.run(tf.global_variables_initializer())
    
    if _EPOCH != 0:
        train_loss, train_accuracy, test_accuracy = train(
            sess, _EPOCH, train_x, train_y, test_x, test_y, 
            models={"relu": relu, "selu": selu})

In [ ]:
# Plot Training Loss, Training Accuracy and Test Accuracy for the three activation functions
plot(train_loss, train_accuracy, test_accuracy)        